In [1]:
from pathlib import Path
import json, joblib
import numpy as np
import pandas as pd

from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, classification_report
)
from sklearn.ensemble import RandomForestClassifier

# paths (relative to notebooks/)
ROOT = Path("..").resolve()
DATA = ROOT / "datasets"
MODELS = ROOT / "models" / "optimized"

print("ROOT:", ROOT)
print("DATA exists:", DATA.exists())
print("MODELS exists:", MODELS.exists())


ROOT: C:\Users\richa\OneDrive\Documents\FYP2
DATA exists: True
MODELS exists: True


In [2]:
# --- Static ---
static_df = pd.read_csv(DATA / "static" / "ransomware_detection.csv")

# --- Behavioral ---
behav_df = pd.read_csv(DATA / "behavioral" / "ugransom_with_label.csv")

print("Static shape :", static_df.shape)
print("Behavior shape:", behav_df.shape)
static_df.head(2), behav_df.head(2)


Static shape : (62485, 18)
Behavior shape: (149043, 15)


(                                       FileName  \
 0      0124e21d-018c-4ce0-92a3-b9e205a76bc0.dll   
 1  05c8318f98a5d301d80000009c316005.vertdll.dll   
 
                             md5Hash  Machine  DebugSize  DebugRVA  \
 0  79755c51e413ed3c6be4635fd729a6e1      332          0         0   
 1  95e19f3657d34a432eada93221b0ea16    34404         84    121728   
 
    MajorImageVersion  MajorOSVersion  ExportRVA  ExportSize  IatVRA  \
 0                  0               4          0           0    8192   
 1                 10              10     126576        4930       0   
 
    MajorLinkerVersion  MinorLinkerVersion  NumberOfSections  \
 0                   8                   0                 3   
 1                  14                  10                 8   
 
    SizeOfStackReserve  DllCharacteristics  ResourceSize  BitcoinAddresses  \
 0             1048576               34112           672                 0   
 1              262144               16864          1024      

In [3]:
# --- STATIC ---
y_static = static_df["Benign"].astype(int)
X_static = static_df.drop(columns=["Benign"], errors="ignore")

# --- BEHAVIORAL ---
# fix occasional typos so your CatBoost features line up
behav_df = behav_df.rename(columns={
    "Protcol": "Protocol",
    "SeddAddress": "SeedAddress",
    "Netflow Bytes": "Netflow_Bytes",
})

if "label" in behav_df.columns:
    y_behav = behav_df["label"].astype(int)
else:
    # Fall back: SS = safe(0), otherwise malicious(1)
    y_behav = (behav_df["Prediction"].astype(str).str.upper() != "SS").astype(int)

X_behav = behav_df.drop(columns=[c for c in ["label", "Prediction"] if c in behav_df.columns])


In [4]:
X_static_train, X_static_test, y_static_train, y_static_test = train_test_split(
    X_static, y_static, test_size=0.2, random_state=42, stratify=y_static
)

X_behav_train, X_behav_test, y_behav_train, y_behav_test = train_test_split(
    X_behav, y_behav, test_size=0.2, random_state=42, stratify=y_behav
)


In [5]:
def load_threshold(path_json):
    with open(path_json, "r") as f:
        j = json.load(f)
    # accept either key; your files store "best_threshold"
    return float(j.get("best_threshold", j.get("threshold", 0.5)))

def align_to_feats(df, feats):
    X = df.copy()
    for c in feats:
        if c not in X.columns:
            X[c] = 0.0
    return X[feats]

# --- Static (XGBoost) ---
xgb_model = joblib.load(MODELS / "static_xgb_tuned.joblib")
static_feats = joblib.load(MODELS / "static_xgb_feature_names.joblib")  # list[str]
static_thr = load_threshold(MODELS / "static_xgb_threshold.json")

# --- Behavioral (CatBoost) ---
cb_model = CatBoostClassifier()
cb_model.load_model(str(MODELS / "behav_catboost_tuned.cbm"))
with open(MODELS / "behav_feature_names.json", "r") as f:
    behav_feats = json.load(f)
behav_thr = load_threshold(MODELS / "behav_threshold.json")

static_thr, behav_thr


(0.78, 0.6741265375003658)

In [6]:
def evaluate(name, y_true, y_prob, thr=0.5):
    y_pred = (y_prob >= thr).astype(int)
    print(f"\n==== {name} (thr={thr:.3f}) ====")
    print("Accuracy :", accuracy_score(y_true, y_pred))
    print("Precision:", precision_score(y_true, y_pred))
    print("Recall   :", recall_score(y_true, y_pred))
    print("F1 Score :", f1_score(y_true, y_pred))
    # For ROC-AUC, use the raw probability
    print("ROC AUC  :", roc_auc_score(y_true, y_prob))
    print("\nConfusion Matrix:\n", confusion_matrix(y_true, y_pred))
    print("\nClassification Report:\n", classification_report(y_true, y_pred))


In [7]:
X_static_test_aligned = align_to_feats(X_static_test, static_feats)
y_prob_xgb = xgb_model.predict_proba(X_static_test_aligned)[:, 1]
evaluate("Static XGBoost", y_static_test, y_prob_xgb, thr=static_thr)



==== Static XGBoost (thr=0.780) ====
Accuracy : 0.0016003840921821237
Precision: 0.00028340654669122857
Recall   : 0.0003687315634218289
F1 Score : 0.00032048714045348933
ROC AUC  : 3.507198632716258e-05

Confusion Matrix:
 [[  18 7055]
 [5422    2]]

Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00      7073
           1       0.00      0.00      0.00      5424

    accuracy                           0.00     12497
   macro avg       0.00      0.00      0.00     12497
weighted avg       0.00      0.00      0.00     12497



In [9]:
import json, joblib
import numpy as np
import pandas as pd
from pathlib import Path
from catboost import CatBoostClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, classification_report
)

ROOT   = Path("..").resolve()
MODELS = ROOT / "models" / "optimized"

def load_threshold(p):
    j = json.load(open(p, "r"))
    return float(j.get("best_threshold", j.get("threshold", 0.5)))

def load_feature_list(path):
    """Return a Python list of feature names from .json or .joblib, whatever the structure is."""
    if str(path).endswith(".joblib"):
        feats = joblib.load(path)
        return list(feats)
    data = json.load(open(path, "r"))
    if isinstance(data, list):
        return data
    # common dict shapes
    for key in ("feature_names", "features", "columns", "schema"):
        if key in data:
            val = data[key]
            return list(val) if isinstance(val, list) else list(val.values()) if isinstance(val, dict) else list(val)
    # last resort: keys of the dict
    return list(data)

def align_to_feats(df, feats):
    X = df.copy()
    for c in feats:
        if c not in X.columns:
            X[c] = 0.0
    return X[feats].copy()

def evaluate(name, y_true_pos1, y_prob_pos1, thr=0.5):
    """y_true_pos1 must be 1 for the SAME class whose prob you pass in y_prob_pos1."""
    y_pred = (y_prob_pos1 >= thr).astype(int)
    print(f"\n==== {name} (thr={thr:.3f}) ====")
    print("Accuracy :", accuracy_score(y_true_pos1, y_pred))
    print("Precision:", precision_score(y_true_pos1, y_pred))
    print("Recall   :", recall_score(y_true_pos1, y_pred))
    print("F1 Score :", f1_score(y_true_pos1, y_pred))
    print("ROC AUC  :", roc_auc_score(y_true_pos1, y_prob_pos1))
    print("\nConfusion Matrix:\n", confusion_matrix(y_true_pos1, y_pred))
    print("\nClassification Report:\n", classification_report(y_true_pos1, y_pred))


In [10]:
# --- STATIC (XGBoost) ---
xgb_model   = joblib.load(MODELS / "static_xgb_tuned.joblib")
static_thr  = load_threshold(MODELS / "static_xgb_threshold.json")
static_feats = load_feature_list(MODELS / "static_xgb_feature_names.joblib")

# --- BEHAV (CatBoost) ---
cb_model = CatBoostClassifier()
cb_model.load_model(str(MODELS / "behav_catboost_tuned.cbm"))
behav_thr   = load_threshold(MODELS / "behav_threshold.json")
behav_feats = load_feature_list(MODELS / "behav_feature_names.json")

static_thr, behav_thr, len(static_feats), len(behav_feats)


(0.78, 0.6741265375003658, 15, 3)

In [11]:
y_mal_true = 1 - y_static_test


In [12]:
# --- STATIC (XGB) ---
X_static_test_aligned = align_to_feats(X_static_test, static_feats)
# Model returns prob for its class "1" which is (from your training) MALICIOUS.
y_prob_mal_xgb = xgb_model.predict_proba(X_static_test_aligned)[:, 1]

# Flip your ground truth: you have 1=benign; we need 1=malicious to match y_prob_mal_xgb
y_static_mal_true = (1 - y_static_test).astype(int)

# Evaluate with threshold tuned for MALICIOUS probability (your file stores 0.78)
evaluate("Static XGBoost (P=malicious)", y_static_mal_true, y_prob_mal_xgb, thr=static_thr)



==== Static XGBoost (P=malicious) (thr=0.780) ====
Accuracy : 0.9983996159078179
Precision: 0.9997165934533088
Recall   : 0.9974551109854376
F1 Score : 0.9985845718329794
ROC AUC  : 0.9999649280136729

Confusion Matrix:
 [[5422    2]
 [  18 7055]]

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      5424
           1       1.00      1.00      1.00      7073

    accuracy                           1.00     12497
   macro avg       1.00      1.00      1.00     12497
weighted avg       1.00      1.00      1.00     12497



In [14]:
import json
from pathlib import Path

schema_path = Path("..") / "models" / "optimized" / "behav_feature_names.json"
with open(schema_path, "r") as f:
    behav_schema_raw = json.load(f)

type(behav_schema_raw), list(behav_schema_raw)[:5] if isinstance(behav_schema_raw, list) else behav_schema_raw.keys()


(dict, dict_keys(['use_cols', 'cat_cols', 'num_cols']))

In [16]:
# 1) Load schema properly
import json
from pathlib import Path

ROOT   = Path("..").resolve()
MODELS = ROOT / "models" / "optimized"
SCHEMA = MODELS / "behav_feature_names.json"

with open(SCHEMA, "r") as f:
    schema = json.load(f)

# Expecting keys: use_cols, cat_cols, num_cols
behav_feat_names = schema["use_cols"]          # ordered list of all features the model expects
behav_cat_names  = schema.get("cat_cols", [])  # categorical features by NAME

# Convert categorical names -> indices within behav_feat_names
name_to_idx = {n: i for i, n in enumerate(behav_feat_names)}
behav_cat_idx = [name_to_idx[n] for n in behav_cat_names if n in name_to_idx]

len(behav_feat_names), behav_cat_names, behav_cat_idx[:10]


(12,
 ['Protocol',
  'Flag',
  'Threats',
  'ProtoFlag',
  'PortBucket',
  'has_BTC',
  'has_USD'],
 [5, 6, 7, 8, 9, 10, 11])

In [17]:
# 2) Align your test frame to the exact order + coerce categoricals to string
import pandas as pd
from catboost import Pool

def align_and_pool(df, feat_names, cat_idx):
    X = df.copy()
    # add any missing columns as 0.0
    for c in feat_names:
        if c not in X.columns:
            X[c] = 0.0
    X = X[feat_names]
    # CatBoost categorical columns must be string/object
    for i in cat_idx:
        col = feat_names[i]
        X[col] = X[col].astype(str)
    return Pool(X, cat_features=cat_idx)

behav_pool_test = align_and_pool(X_behav_test, behav_feat_names, behav_cat_idx)


In [18]:
# 3) Load model, threshold, evaluate
from catboost import CatBoostClassifier
import numpy as np

cb_model = CatBoostClassifier()
cb_model.load_model(str(MODELS / "behav_catboost_tuned.cbm"))

# threshold loader reused
import json
def load_threshold(p):
    j = json.load(open(p, "r"))
    return float(j.get("best_threshold", j.get("threshold", 0.5)))

behav_thr = load_threshold(MODELS / "behav_threshold.json")

# CatBoost returns prob of class 1 in [:,1] by default
y_prob_mal_cb = cb_model.predict_proba(behav_pool_test)[:, 1]

# your y_behav_test already uses 1 = malicious
evaluate("Behavioral CatBoost (P=malicious)", y_behav_test, y_prob_mal_cb, thr=behav_thr)



==== Behavioral CatBoost (P=malicious) (thr=0.674) ====
Accuracy : 0.5424871683048743
Precision: 0.0
Recall   : 0.0
F1 Score : 0.0
ROC AUC  : 0.5641433083066827

Confusion Matrix:
 [[16171     0]
 [13638     0]]

Classification Report:
               precision    recall  f1-score   support

           0       0.54      1.00      0.70     16171
           1       0.00      0.00      0.00     13638

    accuracy                           0.54     29809
   macro avg       0.27      0.50      0.35     29809
weighted avg       0.29      0.54      0.38     29809



c:\Users\richa\OneDrive\Documents\FYP2\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\richa\OneDrive\Documents\FYP2\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\richa\OneDrive\Documents\FYP2\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()

In [21]:
import sys, os
sys.path.append(os.path.abspath(".."))
from src.behav_model import predict_behav
import pandas as pd

sample = X_behav_test.sample(5, random_state=1)
probs, preds, thr, reasons = predict_behav(sample)

print("Threshold:", thr)
print("Predictions:", preds)
print("Probabilities:", probs[:5])
print("Reasons:", reasons)


Threshold: 0.6741265375003658
Predictions: [0 0 0 0 0]
Probabilities: [0.00193079 0.00208744 0.00209444 0.00202799 0.00189672]
Reasons: ['This file’s characteristics look unusual compared to safe files.']


In [23]:
import sys, os
sys.path.append(os.path.abspath(".."))

from src.behav_model import predict_behav


In [24]:
# e.g., take rows with BTC>0 or Port typical of ransomware traffic
sample = X_behav_test.copy()
sample = sample[sample.get("BTC", 0).astype(float) > 0].head(10)  # tweak if your columns differ
probs, preds, thr, reasons = predict_behav(sample)
print("thr:", thr)
print("preds:", preds)
print("probs:", probs[:10])
print("reasons:", reasons)


thr: 0.6741265375003658
preds: [0 0 0 0 0 0 0 0 0 0]
probs: [0.00184869 0.05645192 0.00075953 0.01603126 0.00855485 0.00329618
 0.00076715 0.00498016 0.00013146 0.00546316]
reasons: ['This file’s characteristics look unusual compared to safe files.']


In [25]:
# y_behav_test must be 1=malicious (which you already have)
probs, preds, thr, _ = predict_behav(X_behav_test)
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report

print("Accuracy :", accuracy_score(y_behav_test, preds))
print("Precision:", precision_score(y_behav_test, preds))
print("Recall   :", recall_score(y_behav_test, preds))
print("F1 Score :", f1_score(y_behav_test, preds))
print("ROC AUC  :", roc_auc_score(y_behav_test, probs))
print(confusion_matrix(y_behav_test, preds))
print(classification_report(y_behav_test, preds))


Accuracy : 0.5424871683048743
Precision: 0.0
Recall   : 0.0
F1 Score : 0.0
ROC AUC  : 0.5641433083066827
[[16171     0]
 [13638     0]]
              precision    recall  f1-score   support

           0       0.54      1.00      0.70     16171
           1       0.00      0.00      0.00     13638

    accuracy                           0.54     29809
   macro avg       0.27      0.50      0.35     29809
weighted avg       0.29      0.54      0.38     29809



c:\Users\richa\OneDrive\Documents\FYP2\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\richa\OneDrive\Documents\FYP2\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\richa\OneDrive\Documents\FYP2\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()

In [26]:
import pandas as pd
import numpy as np

def build_behav_features(df: pd.DataFrame) -> pd.DataFrame:
    X = df.copy()

    # Standardize common typos (you already did this once)
    X = X.rename(columns={
        "Protcol": "Protocol",
        "SeddAddress": "SeedAddress",
        "Netflow Bytes": "Netflow_Bytes",
    })

    # Coerce numeric columns safely (silently coerce errors to NaN -> then fill 0)
    for col in ["BTC","USD","Port","Netflow_Bytes","Time","Clusters"]:
        if col in X.columns:
            X[col] = pd.to_numeric(X[col], errors="coerce").fillna(0)

    # ---- engineered features your CatBoost expects ----
    # 1) ProtoFlag: combo of Protocol + Flag as a categorical string
    X["ProtoFlag"] = X.get("Protocol", "").astype(str) + "_" + X.get("Flag", "").astype(str)

    # 2) PortBucket: bucketize port into well-known / registered / dynamic
    if "Port" in X.columns:
        X["PortBucket"] = pd.cut(
            X["Port"].astype(float),
            bins=[-1, 1023, 49151, 65535],
            labels=["well_known","registered","dynamic"]
        ).astype(str)
    else:
        X["PortBucket"] = "unknown"

    # 3) has_BTC / has_USD flags
    X["has_BTC"] = (X.get("BTC", 0).astype(float) > 0).astype(int)
    X["has_USD"] = (X.get("USD", 0).astype(float) > 0).astype(int)

    return X


In [27]:
from pathlib import Path
import json, joblib
from catboost import Pool, CatBoostClassifier

ROOT   = Path("..").resolve()
MODELS = ROOT / "models" / "optimized"

# Load schema used by the trained CatBoost
schema = json.load(open(MODELS / "behav_feature_names.json", "r"))
behav_feat_names = schema["use_cols"]          # ordered list from training
behav_cat_names  = schema.get("cat_cols", [])  # categorical feature names from training
name_to_idx = {n: i for i, n in enumerate(behav_feat_names)}
behav_cat_idx = [name_to_idx[n] for n in behav_cat_names if n in name_to_idx]

def align_and_pool(df, feat_names, cat_idx):
    X = df.copy()
    # ensure all expected feat columns exist
    for c in feat_names:
        if c not in X.columns:
            X[c] = 0.0
    X = X[feat_names].copy()
    # CatBoost categorical columns must be strings/objects
    for i in cat_idx:
        col = feat_names[i]
        X[col] = X[col].astype(str)
    return Pool(X, cat_features=cat_idx)

# Build features on the FULL behavioral dataframe first, then use your split
behav_full_feats = build_behav_features(X_behav)  # your X_behav from earlier split
behav_pool_test  = align_and_pool(behav_full_feats.loc[X_behav_test.index], behav_feat_names, behav_cat_idx)

# Load model + threshold
cb_model  = CatBoostClassifier()
cb_model.load_model(str(MODELS / "behav_catboost_tuned.cbm"))
thr_json  = json.load(open(MODELS / "behav_threshold.json","r"))
behav_thr = float(thr_json.get("best_threshold", thr_json.get("threshold", 0.5)))


In [28]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report
import numpy as np

def evaluate(title, y_true, y_prob, thr):
    y_pred = (y_prob >= thr).astype(int)
    print(f"\n== {title} ==")
    print("Accuracy :", accuracy_score(y_true, y_pred))
    print("Precision:", precision_score(y_true, y_pred, zero_division=0))
    print("Recall   :", recall_score(y_true, y_pred, zero_division=0))
    print("F1 Score :", f1_score(y_true, y_pred, zero_division=0))
    print("ROC AUC  :", roc_auc_score(y_true, y_prob))
    print(confusion_matrix(y_true, y_pred))
    print(classification_report(y_true, y_pred, zero_division=0))

# Get probabilities from CatBoost
y_prob_col1 = cb_model.predict_proba(behav_pool_test)[:, 1]

# Your ground-truth: y_behav_test currently (from earlier) used 1=malicious (based on label or Prediction!=SS).
# Let's evaluate both orientations:
evaluate("Behavior CatBoost assuming prob[:,1] = P(malicious)", y_behav_test, y_prob_col1, behav_thr)
evaluate("Behavior CatBoost assuming prob[:,1] = P(benign) (flip)", y_behav_test, 1.0 - y_prob_col1, 1.0 - behav_thr)



== Behavior CatBoost assuming prob[:,1] = P(malicious) ==
Accuracy : 0.5424871683048743
Precision: 0.5
Recall   : 0.001173192550227306
F1 Score : 0.0023408924652523777
ROC AUC  : 0.5777947101483558
[[16155    16]
 [13622    16]]
              precision    recall  f1-score   support

           0       0.54      1.00      0.70     16171
           1       0.50      0.00      0.00     13638

    accuracy                           0.54     29809
   macro avg       0.52      0.50      0.35     29809
weighted avg       0.52      0.54      0.38     29809


== Behavior CatBoost assuming prob[:,1] = P(benign) (flip) ==
Accuracy : 0.45751283169512563
Precision: 0.4574671726500319
Recall   : 0.9988268074497727
F1 Score : 0.6275250489462167
ROC AUC  : 0.4222052898516441
[[   16 16155]
 [   16 13622]]
              precision    recall  f1-score   support

           0       0.50      0.00      0.00     16171
           1       0.46      1.00      0.63     13638

    accuracy                      

In [29]:
behav_df[["Prediction", "label"]].value_counts().head(10)


Prediction  label
S           0        35251
            1        31129
A           0        24504
SS          0        21101
            1        19001
A           1        18057
Name: count, dtype: int64

In [30]:
behav_df.groupby("Prediction")["label"].value_counts()


Prediction  label
A           0        24504
            1        18057
S           0        35251
            1        31129
SS          0        21101
            1        19001
Name: count, dtype: int64

In [31]:
cb_model.get_params()["class_names"] if "class_names" in cb_model.get_params() else cb_model.classes_ if hasattr(cb_model, "classes_") else None


array([0, 1])

In [32]:
cb_model.get_all_params()["class_names"] if "class_names" in cb_model.get_all_params() else None


[0, 1]

In [34]:
sample_df = X_behav_test.head(5)
sample_pool = align_and_pool(sample_df, behav_feat_names, behav_cat_idx)
cb_model.predict_proba(sample_pool)


array([[9.98151313e-01, 1.84868721e-03],
       [9.43548077e-01, 5.64519225e-02],
       [9.99240472e-01, 7.59528439e-04],
       [9.83968744e-01, 1.60312556e-02],
       [9.91445153e-01, 8.55484690e-03]])

In [35]:
y_prob_mal_cb = 1.0 - cb_model.predict_proba(behav_pool_test)[:, 1]
evaluate("Behavioral CatBoost (P=malicious, flipped)", y_behav_test, y_prob_mal_cb, thr=behav_thr)



== Behavioral CatBoost (P=malicious, flipped) ==
Accuracy : 0.4599282096011272
Precision: 0.4581704456606724
Recall   : 0.9882680744977269
F1 Score : 0.6260829171989316
ROC AUC  : 0.4222052898516441
[[  232 15939]
 [  160 13478]]
              precision    recall  f1-score   support

           0       0.59      0.01      0.03     16171
           1       0.46      0.99      0.63     13638

    accuracy                           0.46     29809
   macro avg       0.53      0.50      0.33     29809
weighted avg       0.53      0.46      0.30     29809



In [3]:
import os
import pandas as pd
import numpy as np

print("=== EXPLORING YOUR DIRECTORY STRUCTURE ===")

# Check what's actually in your datasets/behavioral directory
behavioral_path = 'datasets/behavioral/'
if os.path.exists(behavioral_path):
    print("Files in datasets/behavioral/:")
    for file in os.listdir(behavioral_path):
        print(f"  - {file}")
else:
    print(f"Directory {behavioral_path} does not exist!")

# Check the root directory
print(f"\n=== ROOT DIRECTORY CONTENTS ===")
for item in os.listdir('.'):
    if os.path.isdir(item):
        print(f"📁 {item}/")
    else:
        print(f"📄 {item}")

# Check if there are any CSV files in the entire project
print(f"\n=== SEARCHING FOR BEHAVIORAL CSV FILES ===")
for root, dirs, files in os.walk('.'):
    for file in files:
        if 'behavior' in file.lower() or 'behav' in file.lower() and file.endswith('.csv'):
            print(f"Found: {os.path.join(root, file)}")
        if 'ugransom' in file.lower() and file.endswith('.csv'):
            print(f"Found: {os.path.join(root, file)}")

# Also check your data_processed directory
print(f"\n=== CHECKING data_processed/ ===")
if os.path.exists('data_processed/'):
    for file in os.listdir('data_processed/'):
        print(f"  - {file}")

=== EXPLORING YOUR DIRECTORY STRUCTURE ===
Directory datasets/behavioral/ does not exist!

=== ROOT DIRECTORY CONTENTS ===
📄 01_inspect_datasets.ipynb
📄 02_evaluate_models.ipynb
📄 03_model_optimization.ipynb
📄 04_compare_final_models.ipynb
📄 05_test.ipynb
📁 catboost_info/
📄 sanity_check.ipynb

=== SEARCHING FOR BEHAVIORAL CSV FILES ===

=== CHECKING data_processed/ ===


In [5]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix

# Load the behavioral dataset with the correct path
behav_data = pd.read_csv('../datasets/behavioral/ugransom_with_label.csv')

print("=== DATASET LABEL ANALYSIS ===")
print(f"Dataset shape: {behav_data.shape}")

# Check all potential label columns
label_columns = [col for col in behav_data.columns if 'label' in col.lower() or 'pred' in col.lower() or 'class' in col.lower()]
print(f"Potential label columns: {label_columns}")

for col in label_columns:
    print(f"\n--- {col} ---")
    print(f"Value counts:\n{behav_data[col].value_counts()}")
    print(f"Unique values: {behav_data[col].unique()}")
    print(f"Data type: {behav_data[col].dtype}")

# Check feature columns
print(f"\n=== FEATURE COLUMNS ===")
print(f"Number of features: {len(behav_data.columns)}")
print(f"Feature names: {list(behav_data.columns)}")

# Check for missing values
print(f"\n=== MISSING VALUES ===")
print(behav_data.isnull().sum())

# Sample some rows to understand the data
print(f"\n=== SAMPLE DATA (first 3 rows) ===")
print(behav_data.head(3))

print(f"\n=== DATA TYPES ===")
print(behav_data.dtypes)

=== DATASET LABEL ANALYSIS ===
Dataset shape: (149043, 15)
Potential label columns: ['Prediction', 'label']

--- Prediction ---
Value counts:
Prediction
S     66380
A     42561
SS    40102
Name: count, dtype: int64
Unique values: ['SS' 'A' 'S']
Data type: object

--- label ---
Value counts:
label
0    80856
1    68187
Name: count, dtype: int64
Unique values: [1 0]
Data type: int64

=== FEATURE COLUMNS ===
Number of features: 15
Feature names: ['Time', 'Protcol', 'Flag', 'Family', 'Clusters', 'SeddAddress', 'ExpAddress', 'BTC', 'USD', 'Netflow_Bytes', 'IPaddress', 'Threats', 'Port', 'Prediction', 'label']

=== MISSING VALUES ===
Time             0
Protcol          0
Flag             0
Family           0
Clusters         0
SeddAddress      0
ExpAddress       0
BTC              0
USD              0
Netflow_Bytes    0
IPaddress        0
Threats          0
Port             0
Prediction       0
label            0
dtype: int64

=== SAMPLE DATA (first 3 rows) ===
   Time Protcol Flag    Family

In [6]:
# Check relationship between Prediction and label columns
print("=== LABEL COLUMNS RELATIONSHIP ===")
if 'Prediction' in behav_data.columns and 'label' in behav_data.columns:
    cross_tab = pd.crosstab(behav_data['Prediction'], behav_data['label'])
    print("Cross-tabulation between 'Prediction' and 'label':")
    print(cross_tab)
    
    # Check if they agree
    agreement = (behav_data['Prediction'] == behav_data['label']).mean()
    print(f"\nAgreement between Prediction and label: {agreement:.3f}")

=== LABEL COLUMNS RELATIONSHIP ===
Cross-tabulation between 'Prediction' and 'label':
label           0      1
Prediction              
A           24504  18057
S           35251  31129
SS          21101  19001

Agreement between Prediction and label: 0.000


In [8]:
import os
import json

print("=== EXPLORING MODELS DIRECTORY ===")
models_path = 'models/'
if os.path.exists(models_path):
    print("Files in models/:")
    for root, dirs, files in os.walk(models_path):
        level = root.replace(models_path, '').count(os.sep)
        indent = ' ' * 2 * level
        print(f"{indent}📁 {os.path.basename(root)}/")
        subindent = ' ' * 2 * (level + 1)
        for file in files:
            print(f"{subindent}📄 {file}")
else:
    print("models/ directory does not exist!")

# Check if models are in parent directory
print(f"\n=== CHECKING PARENT DIRECTORY FOR MODELS ===")
parent_models_path = '../models/'
if os.path.exists(parent_models_path):
    print("Files in ../models/:")
    for root, dirs, files in os.walk(parent_models_path):
        level = root.replace(parent_models_path, '').count(os.sep)
        indent = ' ' * 2 * level
        print(f"{indent}📁 {os.path.basename(root)}/")
        subindent = ' ' * 2 * (level + 1)
        for file in files:
            print(f"{subindent}📄 {file}")

=== EXPLORING MODELS DIRECTORY ===
models/ directory does not exist!

=== CHECKING PARENT DIRECTORY FOR MODELS ===
Files in ../models/:
📁 /
📁 models/
  📄 behav_catboost.cbm
  📄 behav_catboost_schema.json
  📄 behav_threshold.json
📁 optimized/
  📄 behav_catboost_tuned.cbm
  📄 behav_feature_names.json
  📄 behav_threshold.json
  📄 static_rf_feature_names.joblib
  📄 static_rf_threshold.json
  📄 static_rf_tuned.joblib
  📄 static_xgb.joblib
  📄 static_xgb_feature_names.joblib
  📄 static_xgb_threshold.json
  📄 static_xgb_tuned.joblib
  📄 _threshold.json


In [9]:
print("=== CRITICAL FINDING: LABEL MISMATCH ===")
print("The 'Prediction' and 'label' columns COMPLETELY DISAGREE!")
print(f"Agreement: {(behav_data['Prediction'] == behav_data['label']).mean():.3f}")

# Let's understand what the Prediction column values mean
print(f"\n=== PREDICTION COLUMN ANALYSIS ===")
print("Prediction value counts:")
print(behav_data['Prediction'].value_counts())

# Check if there's a pattern between Prediction values and the label
print(f"\n=== PREDICTION VS LABEL MAPPING ===")
cross_tab = pd.crosstab(behav_data['Prediction'], behav_data['label'], normalize='index') * 100
print("Percentage distribution of labels for each Prediction value:")
print(cross_tab.round(2))

# Let's see if we can infer the correct mapping
print(f"\n=== ATTEMPTING TO INFER CORRECT LABEL MAPPING ===")

# Common ransomware dataset mappings:
# 'A' = Benign (Normal Activity)
# 'S' = Suspicious 
# 'SS' = Highly Suspicious/Malicious

# Let's check the distribution of known ransomware families
print(f"\n=== RANSOMWARE FAMILY ANALYSIS ===")
family_counts = behav_data['Family'].value_counts()
print("Family distribution:")
print(family_counts)

# Check if WannaCry (known ransomware) correlates with certain Prediction values
wannacry_data = behav_data[behav_data['Family'] == 'WannaCry']
print(f"\nWannaCry samples: {len(wannacry_data)}")
print("WannaCry Prediction distribution:")
print(wannacry_data['Prediction'].value_counts())
print("WannaCry label distribution:")
print(wannacry_data['label'].value_counts())

=== CRITICAL FINDING: LABEL MISMATCH ===
The 'Prediction' and 'label' columns COMPLETELY DISAGREE!
Agreement: 0.000

=== PREDICTION COLUMN ANALYSIS ===
Prediction value counts:
Prediction
S     66380
A     42561
SS    40102
Name: count, dtype: int64

=== PREDICTION VS LABEL MAPPING ===
Percentage distribution of labels for each Prediction value:
label           0      1
Prediction              
A           57.57  42.43
S           53.10  46.90
SS          52.62  47.38

=== ATTEMPTING TO INFER CORRECT LABEL MAPPING ===

=== RANSOMWARE FAMILY ANALYSIS ===
Family distribution:
Family
Locky               25062
SamSam              19657
WannaCry            16110
JigSaw              13712
Flyper              12014
DMALocker           11360
APT                  9730
CryptXXX             9335
Razy                 7862
Globe                7373
EDA2                 6054
TowerWeb             4381
Cryptohitman         4134
NoobCrypt            1248
CryptoLocker          788
CryptoLocker2015      

In [10]:
# Try to find any CatBoost model
print("\n=== SEARCHING FOR CATBOOST MODELS ===")
possible_model_paths = [
    'models/behav_catboost.cbm',
    '../models/behav_catboost.cbm',
    'models/optimized/behav_catboost_tuned.cbm', 
    '../models/optimized/behav_catboost_tuned.cbm',
    'models/behav_catboost_tuned.cbm',
    '../models/behav_catboost_tuned.cbm'
]

for path in possible_model_paths:
    if os.path.exists(path):
        print(f"✅ FOUND: {path}")
        # Try to load it
        try:
            from catboost import CatBoost
            test_model = CatBoost()
            test_model.load_model(path)
            print(f"   Successfully loaded model!")
            # Check model classes
            if hasattr(test_model, 'classes_'):
                print(f"   Model classes: {test_model.classes_}")
        except Exception as e:
            print(f"   Error loading: {e}")
    else:
        print(f"❌ Not found: {path}")


=== SEARCHING FOR CATBOOST MODELS ===
❌ Not found: models/behav_catboost.cbm
❌ Not found: ../models/behav_catboost.cbm
❌ Not found: models/optimized/behav_catboost_tuned.cbm
✅ FOUND: ../models/optimized/behav_catboost_tuned.cbm
   Successfully loaded model!
   Model classes: [0 1]
❌ Not found: models/behav_catboost_tuned.cbm
❌ Not found: ../models/behav_catboost_tuned.cbm


In [11]:
# Based on common ransomware dataset conventions, let's test this hypothesis:
# 'A' = Benign (label should be 0)
# 'S' = Suspicious (label should be 1) 
# 'SS' = Malicious (label should be 1)

print("\n=== TESTING LABEL MAPPING HYPOTHESIS ===")

# Hypothesis: A=Benign(0), S=Malicious(1), SS=Malicious(1)
behav_data['label_mapped'] = behav_data['Prediction'].map({'A': 0, 'S': 1, 'SS': 1})

print("New label mapping:")
print(behav_data[['Prediction', 'label', 'label_mapped']].value_counts().sort_index())

# Check agreement with original label
agreement_new = (behav_data['label'] == behav_data['label_mapped']).mean()
print(f"Agreement with original 'label' column: {agreement_new:.3f}")

# Let's also check the opposite hypothesis
print(f"\n=== TESTING OPPOSITE HYPOTHESIS ===")
behav_data['label_mapped_opposite'] = behav_data['Prediction'].map({'A': 1, 'S': 0, 'SS': 0})
agreement_opposite = (behav_data['label'] == behav_data['label_mapped_opposite']).mean()
print(f"Opposite mapping agreement: {agreement_opposite:.3f}")

# The correct mapping should have high agreement with the original label column
if agreement_new > agreement_opposite:
    print("✅ Hypothesis confirmed: A=Benign(0), S/S=Malicious(1)")
    correct_mapping = {'A': 0, 'S': 1, 'SS': 1}
else:
    print("✅ Opposite hypothesis confirmed: A=Malicious(1), S/S=Benign(0)") 
    correct_mapping = {'A': 1, 'S': 0, 'SS': 0}

print(f"Correct mapping: {correct_mapping}")


=== TESTING LABEL MAPPING HYPOTHESIS ===
New label mapping:
Prediction  label  label_mapped
A           0      0               24504
            1      0               18057
S           0      1               35251
            1      1               31129
SS          0      1               21101
            1      1               19001
Name: count, dtype: int64
Agreement with original 'label' column: 0.501

=== TESTING OPPOSITE HYPOTHESIS ===
Opposite mapping agreement: 0.499
✅ Hypothesis confirmed: A=Benign(0), S/S=Malicious(1)
Correct mapping: {'A': 0, 'S': 1, 'SS': 1}


In [13]:
print("=== ANALYZING SCHEMA VS ACTUAL DATA ===")
print("Schema expected columns:", feature_schema['use_cols'])
print("Actual columns in dataset:", list(behav_data.columns))

# Check which schema columns actually exist
existing_cols = [col for col in feature_schema['use_cols'] if col in behav_data.columns]
missing_cols = [col for col in feature_schema['use_cols'] if col not in behav_data.columns]

print(f"\nExisting columns: {existing_cols}")
print(f"Missing columns: {missing_cols}")

# Let's create the missing engineered features
print("\n=== CREATING MISSING ENGINEERED FEATURES ===")

# Fix typo in original data (Protcol -> Protocol)
behav_data['Protocol'] = behav_data['Protcol']

# Create ProtoFlag: Protocol_Flag combination
behav_data['ProtoFlag'] = behav_data['Protocol'] + '_' + behav_data['Flag']

# Create PortBucket: bucket the port numbers
def bucket_port(port):
    if port <= 1024:
        return 'well_known'
    elif port <= 49151:
        return 'registered'
    else:
        return 'dynamic_private'

behav_data['PortBucket'] = behav_data['Port'].apply(bucket_port)

# Create has_BTC and has_USD
behav_data['has_BTC'] = (behav_data['BTC'] > 0).astype(str)
behav_data['has_USD'] = (behav_data['USD'] > 0).astype(str)

print("Created missing engineered features")
print("New columns:", [col for col in ['Protocol', 'ProtoFlag', 'PortBucket', 'has_BTC', 'has_USD']])

=== ANALYZING SCHEMA VS ACTUAL DATA ===
Schema expected columns: ['Time', 'Clusters', 'Netflow_Bytes', 'BTC', 'USD', 'Protocol', 'Flag', 'Threats', 'ProtoFlag', 'PortBucket', 'has_BTC', 'has_USD']
Actual columns in dataset: ['Time', 'Protcol', 'Flag', 'Family', 'Clusters', 'SeddAddress', 'ExpAddress', 'BTC', 'USD', 'Netflow_Bytes', 'IPaddress', 'Threats', 'Port', 'Prediction', 'label', 'label_mapped', 'label_mapped_opposite', 'y_true_proper']

Existing columns: ['Time', 'Clusters', 'Netflow_Bytes', 'BTC', 'USD', 'Flag', 'Threats']
Missing columns: ['Protocol', 'ProtoFlag', 'PortBucket', 'has_BTC', 'has_USD']

=== CREATING MISSING ENGINEERED FEATURES ===
Created missing engineered features
New columns: ['Protocol', 'ProtoFlag', 'PortBucket', 'has_BTC', 'has_USD']


In [16]:
# 1 = malicious, 0 = benign
y = behav_data["Prediction"].map({"A": 0, "S": 1, "SS": 1}).astype(int)


In [17]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, Pool
import json

# ==== LOAD DATASET ====
df = pd.read_csv("../datasets/behavioral/ugransom_with_label.csv")

# ==== DEFINE TRUE LABELS (Step 0 DONE) ====
# 1 = malicious, 0 = benign
y = df["Prediction"].map({"A": 0, "S": 1, "SS": 1}).astype(int).values

print("Label distribution:", pd.Series(y).value_counts())

# ==== LOAD SCHEMA ====
schema_path = "../models/optimized/behav_feature_names.json"
with open(schema_path, "r") as f:
    schema = json.load(f)

use_cols = schema["use_cols"]
cat_cols = schema["cat_cols"]

# ==== SANITIZE COLUMNS (MATCH behav_model.py) ====
def _sanitize_columns(df):
    rename_map = {
        "Protcol": "Protocol",
        "SeddAddress": "SeedAddress",
        "Netflow Bytes": "Netflow_Bytes",
    }
    return df.rename(columns={c: rename_map.get(c, c) for c in df.columns})

# ==== ENGINEER FEATURES (MATCH behav_model.py) ====
def _engineer_features(df):
    X = df.copy()

    for col in ["BTC", "USD", "Port", "Netflow_Bytes", "Time", "Clusters"]:
        if col in X.columns:
            X[col] = pd.to_numeric(X[col], errors="coerce").fillna(0)

    # ProtoFlag
    X["ProtoFlag"] = X.get("Protocol", "").astype(str) + "_" + X.get("Flag", "").astype(str)

    # PortBucket
    if "Port" in X.columns:
        X["PortBucket"] = pd.cut(
            X["Port"].astype(float),
            bins=[-1, 1023, 49151, 65535],
            labels=["well_known", "registered", "dynamic"]
        ).astype(str)
    else:
        X["PortBucket"] = "unknown"

    # Has BTC / USD
    X["has_BTC"] = (X.get("BTC", 0).astype(float) > 0).astype(int)
    X["has_USD"] = (X.get("USD", 0).astype(float) > 0).astype(int)

    return X

dfx = _sanitize_columns(df)
dfx = _engineer_features(dfx)

# ==== ALIGN TO FEATURE ORDER ====
X = dfx[use_cols].copy()

# CAST CATEGORICALS
for col in cat_cols:
    if col in X.columns:
        X[col] = X[col].astype(str)

# ==== BUILD POOL ====
cat_idx = [use_cols.index(c) for c in cat_cols if c in use_cols]
pool = Pool(X, cat_features=cat_idx)

# ==== LOAD MODEL ====
model = CatBoostClassifier()
model.load_model("../models/optimized/behav_catboost_tuned.cbm")

# ==== GET RAW PROBABILITY (MODEL'S CLASS 1) ====
p1 = model.predict_proba(pool)[:, 1]

# ==== ORIENTATION CHECK ====
mean_p1_y1 = p1[y == 1].mean()
mean_p1_y0 = p1[y == 0].mean()

print("\n=== ORIENTATION AUDIT ===")
print(f"Mean p1 when MALICIOUS (y=1): {mean_p1_y1:.4f}")
print(f"Mean p1 when BENIGN    (y=0): {mean_p1_y0:.4f}")

if mean_p1_y1 > mean_p1_y0:
    print("\n🎯 RESULT: Class 1 = MALICIOUS → POS_CLASS_IS_BENIGN = False")
else:
    print("\n🎯 RESULT: Class 1 = BENIGN → POS_CLASS_IS_BENIGN = True   (use 1 - p1)")


Label distribution: 1    106482
0     42561
Name: count, dtype: int64

=== ORIENTATION AUDIT ===
Mean p1 when MALICIOUS (y=1): 0.0276
Mean p1 when BENIGN    (y=0): 0.0344

🎯 RESULT: Class 1 = BENIGN → POS_CLASS_IS_BENIGN = True   (use 1 - p1)


In [18]:
import pandas as pd
import numpy as np
import json
from pathlib import Path
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

ROOT = Path("..").resolve()
DATA = ROOT / "datasets" / "behavioral" / "ugransom_with_label.csv"
MODELS = ROOT / "models" / "optimized"
SCHEMA = MODELS / "behav_feature_names.json"
OUT_THR = MODELS / "behav_threshold.json"

# ---- 1. Load data & define true labels (1 = malicious, 0 = benign) ----
df = pd.read_csv(DATA)
y = df["Prediction"].map({"A": 0, "S": 1, "SS": 1}).astype(int).values
print("Label distribution:\n", pd.Series(y).value_counts())

# ---- 2. Schema & feature pipeline (same as behav_model.py) ----
with open(SCHEMA, "r") as f:
    schema = json.load(f)

use_cols = schema["use_cols"]
cat_cols = schema.get("cat_cols", [])

def _sanitize_columns(df):
    rename_map = {
        "Protcol": "Protocol",
        "SeddAddress": "SeedAddress",
        "Netflow Bytes": "Netflow_Bytes",
    }
    return df.rename(columns={c: rename_map.get(c, c) for c in df.columns})

def _engineer_features(df):
    X = df.copy()
    for col in ["BTC","USD","Port","Netflow_Bytes","Time","Clusters"]:
        if col in X.columns:
            X[col] = pd.to_numeric(X[col], errors="coerce").fillna(0)
    X["ProtoFlag"] = X.get("Protocol", "").astype(str) + "_" + X.get("Flag", "").astype(str)
    if "Port" in X.columns:
        X["PortBucket"] = pd.cut(
            X["Port"].astype(float),
            bins=[-1, 1023, 49151, 65535],
            labels=["well_known","registered","dynamic"]
        ).astype(str)
    else:
        X["PortBucket"] = "unknown"
    X["has_BTC"] = (X.get("BTC", 0).astype(float) > 0).astype(int)
    X["has_USD"] = (X.get("USD", 0).astype(float) > 0).astype(int)
    return X

def _align_and_pool(df, feats, cat_cols):
    X = df.copy()
    for c in feats:
        if c not in X.columns:
            X[c] = 0.0
    X = X[feats].copy()
    cat_idx = []
    for col in cat_cols:
        if col in feats:
            idx = feats.index(col)
            cat_idx.append(idx)
            X[col] = X[col].astype(str)
    return Pool(X, cat_features=cat_idx)

dfx = _sanitize_columns(df)
dfx = _engineer_features(dfx)
pool = _align_and_pool(dfx, use_cols, cat_cols)

# ---- 3. Load model & compute P(malicious) ----
cb = CatBoostClassifier()
cb.load_model(str(MODELS / "behav_catboost_tuned.cbm"))

p1 = cb.predict_proba(pool)[:, 1]      # this is P(benign), we just confirmed
p_mal = 1.0 - p1                       # malicious probability

# AUC just to see how separable it is
auc = roc_auc_score(y, p_mal)
print(f"\nROC-AUC (malicious as positive): {auc:.4f}")

# ---- 4. Threshold sweep to maximize F1 ----
best_f1 = -1.0
best_thr = 0.5
records = []

for thr in np.linspace(0.05, 0.95, 19):
    y_pred = (p_mal >= thr).astype(int)
    prec = precision_score(y, y_pred, zero_division=0)
    rec  = recall_score(y, y_pred, zero_division=0)
    f1   = f1_score(y, y_pred, zero_division=0)
    records.append((thr, prec, rec, f1))
    if f1 > best_f1:
        best_f1 = f1
        best_thr = thr

print("\nTop 5 thresholds by F1:")
for thr, prec, rec, f1 in sorted(records, key=lambda x: x[3], reverse=True)[:5]:
    print(f"thr={thr:.3f}  P={prec:.3f}  R={rec:.3f}  F1={f1:.3f}")

print(f"\nChosen threshold (max F1): {best_thr:.3f} with F1={best_f1:.3f}")

# ---- 5. Save tuned threshold for malicious class ----
out = {
    "best_threshold_malicious": float(best_thr),
    "orientation": "p_malicious = 1 - p1"  # because POS_CLASS_IS_BENIGN = True
}
with open(OUT_THR, "w") as f:
    json.dump(out, f, indent=2)

print(f"\nSaved tuned threshold to {OUT_THR}")


Label distribution:
 1    106482
0     42561
Name: count, dtype: int64

ROC-AUC (malicious as positive): 0.4787

Top 5 thresholds by F1:
thr=0.400  P=0.718  R=0.999  F1=0.835
thr=0.350  P=0.716  R=1.000  F1=0.835
thr=0.500  P=0.717  R=0.996  F1=0.834
thr=0.300  P=0.715  R=1.000  F1=0.834
thr=0.450  P=0.717  R=0.996  F1=0.834

Chosen threshold (max F1): 0.400 with F1=0.835

Saved tuned threshold to C:\Users\richa\OneDrive\Documents\FYP2\models\optimized\behav_threshold.json


In [19]:
import pandas as pd
import numpy as np
import json
from pathlib import Path
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, classification_report
)

ROOT   = Path("..").resolve()
DATA   = ROOT / "datasets" / "behavioral" / "ugransom_with_label.csv"
MODELS = ROOT / "models" / "optimized"
SCHEMA = MODELS / "behav_feature_names.json"
THR_JS = MODELS / "behav_threshold.json"

# ---- 1. Load data & label (1 = malicious, 0 = benign) ----
df = pd.read_csv(DATA)
y = df["Prediction"].map({"A": 0, "S": 1, "SS": 1}).astype(int).values
print("Label distribution:\n", pd.Series(y).value_counts())

# ---- 2. Load schema ----
with open(SCHEMA, "r") as f:
    schema = json.load(f)

use_cols = schema["use_cols"]
cat_cols = schema.get("cat_cols", [])

# ---- 3. Same feature pipeline as behav_model.py ----
def _sanitize_columns(df):
    rename_map = {
        "Protcol": "Protocol",
        "SeddAddress": "SeedAddress",
        "Netflow Bytes": "Netflow_Bytes",
    }
    return df.rename(columns={c: rename_map.get(c, c) for c in df.columns})

def _engineer_features(df):
    X = df.copy()
    for col in ["BTC","USD","Port","Netflow_Bytes","Time","Clusters"]:
        if col in X.columns:
            X[col] = pd.to_numeric(X[col], errors="coerce").fillna(0)

    X["ProtoFlag"] = X.get("Protocol", "").astype(str) + "_" + X.get("Flag", "").astype(str)

    if "Port" in X.columns:
        X["PortBucket"] = pd.cut(
            X["Port"].astype(float),
            bins=[-1, 1023, 49151, 65535],
            labels=["well_known","registered","dynamic"]
        ).astype(str)
    else:
        X["PortBucket"] = "unknown"

    X["has_BTC"] = (X.get("BTC", 0).astype(float) > 0).astype(int)
    X["has_USD"] = (X.get("USD", 0).astype(float) > 0).astype(int)

    return X

def _align_and_pool(df, feats, cat_cols):
    X = df.copy()
    for c in feats:
        if c not in X.columns:
            X[c] = 0.0
    X = X[feats].copy()
    cat_idx = []
    for col in cat_cols:
        if col in feats:
            idx = feats.index(col)
            cat_idx.append(idx)
            X[col] = X[col].astype(str)
    return X, cat_idx

dfx = _sanitize_columns(df)
dfx = _engineer_features(dfx)
X_df, cat_idx = _align_and_pool(dfx, use_cols, cat_cols)
pool = Pool(X_df, cat_features=cat_idx)

# ---- 4. Load model and tuned threshold ----
cb = CatBoostClassifier()
cb.load_model(str(MODELS / "behav_catboost_tuned.cbm"))

with open(THR_JS, "r") as f:
    tjson = json.load(f)

thr = float(tjson.get("best_threshold_malicious",
                      tjson.get("best_threshold", tjson.get("threshold", 0.5))))
print(f"\nUsing threshold: {thr:.3f}")

# We already know: POS_CLASS_IS_BENIGN = True  → p1 = P(benign)
p1 = cb.predict_proba(pool)[:, 1]
p_mal = 1.0 - p1   # P(malicious)

# ---- 5. Final metrics ----
y_pred = (p_mal >= thr).astype(int)

print("\n=== FINAL BEHAVIORAL MODEL PERFORMANCE ===")
print("Accuracy :", accuracy_score(y, y_pred))
print("Precision:", precision_score(y, y_pred))
print("Recall   :", recall_score(y, y_pred))
print("F1 Score :", f1_score(y, y_pred))
print("ROC AUC  :", roc_auc_score(y, p_mal))

print("\nConfusion Matrix:\n", confusion_matrix(y, y_pred))
print("\nClassification Report:\n", classification_report(y, y_pred,
                                                        target_names=["Benign", "Malicious"]))


Label distribution:
 1    106482
0     42561
Name: count, dtype: int64

Using threshold: 0.400

=== FINAL BEHAVIORAL MODEL PERFORMANCE ===
Accuracy : 0.71832960957576
Precision: 0.7175574578889496
Recall   : 0.9989481790349543
F1 Score : 0.8351883040660173
ROC AUC  : 0.47872175319702537

Confusion Matrix:
 [[   692  41869]
 [   112 106370]]

Classification Report:
               precision    recall  f1-score   support

      Benign       0.86      0.02      0.03     42561
   Malicious       0.72      1.00      0.84    106482

    accuracy                           0.72    149043
   macro avg       0.79      0.51      0.43    149043
weighted avg       0.76      0.72      0.61    149043



In [21]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import mutual_info_classif

# === 1. LOAD DATA ===
df = pd.read_csv('../datasets/behavioral/ugransom_with_label.csv')

# === 2. CORRECT LABEL MAPPING (GROUND TRUTH) ===
df['y_true'] = df['Prediction'].map({'A': 0, 'S': 1, 'SS': 1}).astype(int)

# === 3. Feature engineering (MINIMUM version, consistent with training schema) ===
df['Protocol'] = df['Protcol']  # fix typo
df['ProtoFlag'] = df['Protocol'].astype(str) + "_" + df['Flag'].astype(str)

# Port bucket
df['PortBucket'] = df['Port'].apply(
    lambda x: 'well_known' if x <= 1024 else
              'registered' if x <= 49151 else
              'dynamic'
)

# BTC/USD presence
df['has_BTC'] = (df['BTC'] > 0).astype(str)
df['has_USD'] = (df['USD'] > 0).astype(str)

# === 4. Feature list (match your schema) ===
feature_cols = [
    'Time', 'Clusters', 'Netflow_Bytes', 'BTC', 'USD',
    'Protocol', 'Flag', 'Threats', 'ProtoFlag', 'PortBucket',
    'has_BTC', 'has_USD'
]

categorical_cols = [
    'Protocol', 'Flag', 'Threats', 'ProtoFlag', 'PortBucket',
    'has_BTC', 'has_USD'
]

# === 5. Build X and y ===
X = df[feature_cols].copy()
y = df['y_true']

# Convert categorical to string for one-hot encoding
for col in categorical_cols:
    X[col] = X[col].astype(str)

# === 6. One-hot encode ===
X_encoded = pd.get_dummies(X, columns=categorical_cols)

# === 7. Mutual Information Analysis ===
mi_scores = mutual_info_classif(X_encoded, y, random_state=42)

feature_importance = pd.DataFrame({
    'feature': X_encoded.columns,
    'importance': mi_scores
}).sort_values('importance', ascending=False)

print("=== FEATURE IMPORTANCE (TOP 10) ===")
print(feature_importance.head(10))
print("\nHighest MI Score:", feature_importance['importance'].max())


=== FEATURE IMPORTANCE (TOP 10) ===
               feature  importance
4                  USD    0.374426
2        Netflow_Bytes    0.253348
3                  BTC    0.249964
1             Clusters    0.219207
36   ProtoFlag_TCP_APS    0.141413
12            Flag_APS    0.135315
17   Threats_Blacklist    0.035903
37  ProtoFlag_TCP_APSF    0.032941
13           Flag_APSF    0.032813
11          Flag_APRSF    0.032293

Highest MI Score: 0.3744262901096429


In [9]:
import sys
from pathlib import Path

ROOT = Path("..").resolve()  # Go up one folder from /notebooks
sys.path.append(str(ROOT))

print("Project root added:", ROOT)


Project root added: C:\Users\richa\OneDrive\Documents\FYP2


In [10]:
from src.behav_model import predict_behav


In [14]:
import pandas as pd
from pathlib import Path
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report

from src.behav_model import predict_behav

# Correct dataset path
DATA = Path("..") / "datasets" / "behavioral" / "ugransom_with_label.csv"

print("Does file exist?", DATA.exists())

# Load data
df = pd.read_csv(DATA)

# Ground truth
y_true = df["Prediction"].map({"A": 0, "S": 1, "SS": 1}).astype(int).values

# Predict
p_mal, preds, thr, reasons = predict_behav(df)

print("Threshold used:", thr)

# Metrics
print("\n=== BEHAVIORAL MODEL (predict_behav) – FULL DATA ===")
print("Accuracy :", accuracy_score(y_true, preds))
print("Precision:", precision_score(y_true, preds))
print("Recall   :", recall_score(y_true, preds))
print("F1 Score :", f1_score(y_true, preds))
print("ROC AUC  :", roc_auc_score(y_true, p_mal))

print("\nClassification report:")
print(classification_report(y_true, preds, target_names=["Benign", "Malicious"]))


Does file exist? True
Threshold used: 0.35

=== BEHAVIORAL MODEL (predict_behav) – FULL DATA ===
Accuracy : 0.9872318726810384
Precision: 0.9852178351041618
Recall   : 0.9970887098288913
F1 Score : 0.9911177286029677
ROC AUC  : 0.9994965771478197

Classification report:
              precision    recall  f1-score   support

      Benign       0.99      0.96      0.98     42561
   Malicious       0.99      1.00      0.99    106482

    accuracy                           0.99    149043
   macro avg       0.99      0.98      0.98    149043
weighted avg       0.99      0.99      0.99    149043



In [16]:
# Take a small random sample to inspect
sample = df.sample(10, random_state=1).copy()

# Ground truth
y_sample = sample["Prediction"].map({"A": 0, "S": 1, "SS": 1}).astype(int).values

# Model predictions
p_sample, preds_sample, thr_used, reasons_sample = predict_behav(sample)

# Use 'Protcol' (raw name), not 'Protocol'
out = sample[["Time", "Clusters", "Netflow_Bytes", "BTC", "USD", "Protcol", "Flag", "Threats", "Prediction"]].copy()
out["y_true"] = y_sample
out["p_malicious"] = p_sample
out["pred"] = preds_sample

print("Threshold used:", thr_used)
out


Threshold used: 0.35


,Time,Clusters,Netflow_Bytes,BTC,USD,Protcol,Flag,Threats,Prediction,y_true,p_malicious,pred
47404,14,1,3506,22,18406,ICMP,R,Blacklist,S,1,0.999978,1
130868,13,2,102,8,870,TCP,A,SSH,SS,1,0.994055,1
35073,20,2,197,12,84388,TCP,APSF,SSH,SS,1,0.996904,1
66361,22,1,10308,11,21345,ICMP,AF,Bonet,S,1,0.925107,1
51057,1,1,46,13,11816,TCP,APS,Bonet,S,1,0.997519,1
109079,16,1,1009,3,621,UDP,ASF,UDP Scan,SS,1,0.999871,1
94951,26,1,1658,3,5444,TCP,APSF,Bonet,SS,1,0.999971,1
77569,2,1,2626,30,1062,ICMP,ASF,DoS,SS,1,0.998556,1
42233,1,1,1501,11,18454,UDP,ARF,Blacklist,S,1,0.999985,1
110686,56,10,453,316,650,ICMP,R,NerisBonet,S,1,0.999808,1


In [17]:
from src.behav_model import _sanitize_columns, _engineer_features


In [18]:
sample = df.sample(10, random_state=1).copy()
y_sample = sample["Prediction"].map({"A": 0, "S": 1, "SS": 1}).astype(int).values

p_sample, preds_sample, thr_used, reasons_sample = predict_behav(sample)

# Apply same cleaning + feature engineering as the model
sample_clean = _sanitize_columns(sample.copy())
sample_feat = _engineer_features(sample_clean)

# Now 'Protocol', 'ProtoFlag', 'PortBucket', 'has_BTC', 'has_USD' all exist
out = sample_feat[[
    "Time", "Clusters", "Netflow_Bytes", "BTC", "USD",
    "Protocol", "Flag", "Threats", "ProtoFlag", "PortBucket",
    "has_BTC", "has_USD"
]].copy()

out["Prediction_raw"] = sample["Prediction"].values
out["y_true"] = y_sample
out["p_malicious"] = p_sample
out["pred"] = preds_sample

print("Threshold used:", thr_used)
out


Threshold used: 0.35


,Time,Clusters,Netflow_Bytes,BTC,USD,Protocol,Flag,Threats,ProtoFlag,PortBucket,has_BTC,has_USD,Prediction_raw,y_true,p_malicious,pred
47404,14,1,3506,22,18406,ICMP,R,Blacklist,ICMP_R,registered,1,1,S,1,0.999978,1
130868,13,2,102,8,870,TCP,A,SSH,TCP_A,registered,1,1,SS,1,0.994055,1
35073,20,2,197,12,84388,TCP,APSF,SSH,TCP_APSF,registered,1,1,SS,1,0.996904,1
66361,22,1,10308,11,21345,ICMP,AF,Bonet,ICMP_AF,registered,1,1,S,1,0.925107,1
51057,1,1,46,13,11816,TCP,APS,Bonet,TCP_APS,registered,1,1,S,1,0.997519,1
109079,16,1,1009,3,621,UDP,ASF,UDP Scan,UDP_ASF,registered,1,1,SS,1,0.999871,1
94951,26,1,1658,3,5444,TCP,APSF,Bonet,TCP_APSF,registered,1,1,SS,1,0.999971,1
77569,2,1,2626,30,1062,ICMP,ASF,DoS,ICMP_ASF,registered,1,1,SS,1,0.998556,1
42233,1,1,1501,11,18454,UDP,ARF,Blacklist,UDP_ARF,registered,1,1,S,1,0.999985,1
110686,56,10,453,316,650,ICMP,R,NerisBonet,ICMP_R,registered,1,1,S,1,0.999808,1


In [19]:
from src.app import combine_model_outputs
import pandas as pd
from pathlib import Path

ROOT = Path("..").resolve()
DATA = ROOT / "datasets" / "behavioral" / "ugransom_with_label.csv"

df = pd.read_csv(DATA)
sample = df.sample(50, random_state=0)  # just a small batch

final_prob, verdict, debug_info = combine_model_outputs(sample)
final_prob, verdict, debug_info


(0.8235946018613538,
 'Malicious',
 {'behavioral': {'threshold': 0.35, 'mean_probability': 0.5600739615949893},
  'static': {'threshold': 0.78, 'mean_probability': 0.9992750287055969},
  'risk_band': 'High Risk',
  'reasons': ['This file’s characteristics look unusual compared to safe files.']})

In [20]:
import pandas as pd
from pathlib import Path

# Project root (you already used this)
ROOT = Path("..").resolve()

# --- Load behavioural dataset ---
behav_path = ROOT / "datasets" / "behavioral" / "ugransom_with_label.csv"
behav_df = pd.read_csv(behav_path)

# Check label distribution just to be sure
behav_df["Prediction"].value_counts()


Prediction
S     66380
A     42561
SS    40102
Name: count, dtype: int64

In [21]:
# Take 5 benign behavioural rows
behav_benign = behav_df[behav_df["Prediction"] == "A"].sample(5, random_state=42).copy()

# OPTIONAL: rename columns to the nicer ones (UI accepts both, thanks to _sanitize_columns)
behav_benign = behav_benign.rename(columns={
    "Protcol": "Protocol",
    "SeddAddress": "SeedAddress",
    "Netflow Bytes": "Netflow_Bytes",
})

# Save demo CSV next to your notebook (or anywhere you like)
out_path_behav = ROOT / "notebooks" / "demo_behav_benign.csv"
behav_benign.to_csv(out_path_behav, index=False)

out_path_behav


WindowsPath('C:/Users/richa/OneDrive/Documents/FYP2/notebooks/demo_behav_benign.csv')

In [22]:
from src.behav_model import predict_behav

p_mal, preds, thr, reasons = predict_behav(behav_benign)
print("Threshold:", thr)
print("Probabilities:", p_mal)
print("Predictions:", preds)


Threshold: 0.35
Probabilities: [2.72711652e-05 8.15158193e-05 3.35172506e-05 2.55376916e-04
 1.15100937e-04]
Predictions: [0 0 0 0 0]


In [23]:
# --- Load static dataset ---
static_path = ROOT / "datasets" / "static" / "ransomware_detection.csv"  # <-- change filename if needed
static_df = pd.read_csv(static_path)

# Check the Benign column
static_df["Benign"].value_counts()


Benign
0    35367
1    27118
Name: count, dtype: int64

In [24]:
# Take 5 malicious static rows
static_malicious = static_df[static_df["Benign"] == 0].sample(5, random_state=42).copy()

# Save demo CSV
out_path_static = ROOT / "demo_static_malicious.csv"
static_malicious.to_csv(out_path_static, index=False)

out_path_static


WindowsPath('C:/Users/richa/OneDrive/Documents/FYP2/demo_static_malicious.csv')

In [25]:
from src.static_model import predict_static

p_mal_s, preds_s, thr_s, reasons_s = predict_static(static_malicious)
print("Static threshold:", thr_s)
print("Probabilities:", p_mal_s)
print("Predictions:", preds_s)


Static threshold: 0.78
Probabilities: [0.99939466 0.99990916 0.99992895 0.99954957 0.9988611 ]
Predictions: [1 1 1 1 1]


In [26]:
import pandas as pd
from pathlib import Path
from src.app import combine_model_outputs

ROOT = Path("..").resolve()
csv_path = ROOT / "demo_behav_benign.csv"   # adjust if needed
df = pd.read_csv(csv_path)

final_prob, verdict, debug_info = combine_model_outputs(df)
print("Final prob:", final_prob)
print("Verdict:", verdict)
print("Debug:", debug_info)


Final prob: 0.38445585565899604
Verdict: Safe
Debug: {'behavioral': {'threshold': 0.35, 'mean_probability': 0.9458645901241598}, 'static': {'threshold': 0.78, 'mean_probability': 0.010183366015553474}, 'risk_band': 'Caution', 'reasons': ['This file’s characteristics look unusual compared to safe files.']}


In [27]:
import pandas as pd
from pathlib import Path

from src.behav_model import predict_behav
from src.static_model import predict_static

ROOT = Path("..").resolve()
DATA = ROOT / "datasets" / "behavioral" / "ugransom_with_label.csv"

df = pd.read_csv(DATA)
print(df.shape)


(149043, 15)


In [28]:
# Behavioral probabilities (P(malicious))
p_behav, _, behav_thr, _ = predict_behav(df)
print("Behavioral threshold:", behav_thr)

# Static probabilities (P(malicious))
p_static, _, static_thr, _ = predict_static(df)
print("Static threshold:", static_thr)

# Attach to dataframe
df["p_behav"] = p_behav
df["p_static"] = p_static

df[["p_behav", "p_static"]].describe()


Behavioral threshold: 0.35
Static threshold: 0.78


,p_behav,p_static
count,149043.000000,149043.000000
mean,0.714475,0.999275
std,0.439820,0.000000
min,0.000001,0.999275
25%,0.026198,0.999275
50%,0.999652,0.999275
75%,0.999961,0.999275
max,0.999999,0.999275


In [29]:
B_THR = 0.35      # behavioral threshold
S_THR = 0.78      # static threshold

behav_safe  = df["p_behav"] < B_THR
behav_mal   = df["p_behav"] >= B_THR

static_safe = df["p_static"] < S_THR      # will be all False (empty)
static_mal  = df["p_static"] >= S_THR     # will be all True
print("static_safe rows:", static_safe.sum())
print("static_mal rows :", static_mal.sum())


static_safe rows: 0
static_mal rows : 149043


In [33]:
behav_cols = [
    "Time", "Protcol", "Flag", "Family", "Clusters",
    "SeddAddress", "ExpAddress", "BTC", "USD",
    "Netflow_Bytes", "IPaddress", "Threats", "Port",
    "Prediction"
]

def make_demo(name, mask, n_rows=5):
    demo = df.loc[mask, behav_cols].head(n_rows).copy()
    print(f"{name} – picked {demo.shape[0]} rows")
    display(demo)
    
    out_path = ROOT / name
    demo.to_csv(out_path, index=False)
    print("Saved to:", out_path)
    return demo


In [34]:
def make_demo(name, mask, n_rows=5):
    demo = df.loc[mask, behav_cols].head(n_rows).copy()
    print(f"{name} – picked {demo.shape[0]} rows")
    display(demo)
    
    out_path = ROOT / name
    demo.to_csv(out_path, index=False)
    print("Saved to:", out_path)
    return demo


In [35]:
mask_ms = static_mal & behav_safe   # effectively just behav_safe

demo_ms = make_demo(
    "demo_fusion_staticMal_behavSafe.csv",
    mask_ms
)

mask_mm = static_mal & behav_mal    # effectively behav_mal

demo_mm = make_demo(
    "demo_fusion_staticMal_behavMal.csv",
    mask_mm
)


demo_fusion_staticMal_behavSafe.csv – picked 5 rows


,Time,Protcol,Flag,Family,Clusters,SeddAddress,ExpAddress,BTC,USD,Netflow_Bytes,IPaddress,Threats,Port,Prediction
710,41,TCP,APS,TowerWeb,1,1DA11mPS,1BonuSr7,2,1,444,C,Spam,5062,A
711,45,TCP,APS,TowerWeb,1,1DA11mPS,1BonuSr7,2,1,465,C,Spam,5062,A
712,49,TCP,APS,TowerWeb,1,1DA11mPS,1BonuSr7,2,1,486,C,Spam,5062,A
713,53,TCP,APS,TowerWeb,1,1DA11mPS,1BonuSr7,2,1,507,C,Spam,5062,A
714,57,TCP,APS,TowerWeb,1,1DA11mPS,1BonuSr7,2,1,528,C,Spam,5062,A


Saved to: C:\Users\richa\OneDrive\Documents\FYP2\demo_fusion_staticMal_behavSafe.csv
demo_fusion_staticMal_behavMal.csv – picked 5 rows


,Time,Protcol,Flag,Family,Clusters,SeddAddress,ExpAddress,BTC,USD,Netflow_Bytes,IPaddress,Threats,Port,Prediction
0,50,TCP,A,WannaCry,1,1DA11mPS,1BonuSr7,1,500,5,A,Bonet,5061,SS
1,40,TCP,A,WannaCry,1,1DA11mPS,1BonuSr7,1,504,8,A,Bonet,5061,SS
2,30,TCP,A,WannaCry,1,1DA11mPS,1BonuSr7,1,508,7,A,Bonet,5061,SS
3,20,TCP,A,WannaCry,1,1DA11mPS,1BonuSr7,1,512,15,A,Bonet,5061,SS
4,57,TCP,A,WannaCry,1,1DA11mPS,1BonuSr7,1,516,9,A,Bonet,5061,SS


Saved to: C:\Users\richa\OneDrive\Documents\FYP2\demo_fusion_staticMal_behavMal.csv


In [36]:
from src.app import combine_model_outputs

for name in [
    "demo_fusion_staticMal_behavSafe.csv",
    "demo_fusion_staticMal_behavMal.csv",
]:
    path = ROOT / name
    d = pd.read_csv(path)
    final_prob, verdict, debug = combine_model_outputs(d)
    print("\n==", name, "==")
    print("Final prob :", round(final_prob, 3), "→", verdict)
    print("Static prob:", round(debug["static"]["mean_probability"], 3))
    print("Behav prob :", round(debug["behavioral"]["mean_probability"], 3))
    print("Band       :", debug["risk_band"])



== demo_fusion_staticMal_behavSafe.csv ==
Final prob : 0.6 → Malicious
Static prob: 0.999
Behav prob : 0.0
Band       : Caution

== demo_fusion_staticMal_behavMal.csv ==
Final prob : 0.977 → Malicious
Static prob: 0.999
Behav prob : 0.944
Band       : High Risk


In [1]:
import os
import json
from pathlib import Path

project_root = Path(".").resolve()

summary = {}

for root, dirs, files in os.walk(project_root):
    rel = os.path.relpath(root, project_root)
    summary[rel] = {"dirs": dirs, "files": files}

import pprint
pprint.pprint(summary)


{'.': {'dirs': ['catboost_info'],
       'files': ['01_inspect_datasets.ipynb',
                 '02_evaluate_models.ipynb',
                 '03_model_optimization.ipynb',
                 '04_compare_final_models.ipynb',
                 '05_test.ipynb',
                 '06_train_behav_lgm.ipynb',
                 'sanity_check.ipynb']},
 'catboost_info': {'dirs': ['learn', 'test', 'tmp'],
                   'files': ['catboost_training.json',
                             'learn_error.tsv',
                             'test_error.tsv',
                             'time_left.tsv']},
 'catboost_info\\learn': {'dirs': [], 'files': ['events.out.tfevents']},
 'catboost_info\\test': {'dirs': [], 'files': ['events.out.tfevents']},
 'catboost_info\\tmp': {'dirs': [], 'files': []}}


In [4]:
from pathlib import Path
import joblib
import os

ROOT = Path("..").resolve()
OPT = ROOT / "models" / "optimized"

feature_path = OPT / "static_xgb_feature_names.joblib"
feats = joblib.load(feature_path)

print("Number of features:", len(feats))
for f in feats:
    print(f)


Number of features: 15
Machine
DebugSize
DebugRVA
MajorImageVersion
MajorOSVersion
ExportRVA
ExportSize
IatVRA
MajorLinkerVersion
MinorLinkerVersion
NumberOfSections
SizeOfStackReserve
DllCharacteristics
ResourceSize
BitcoinAddresses


In [2]:
from pathlib import Path
import pandas as pd

ROOT = Path("..").resolve()   # <-- remove .parent
print(ROOT)                   # should print ...\Documents\FYP2


C:\Users\richa\OneDrive\Documents\FYP2


In [3]:
data_path = ROOT / "datasets" / "static" / "ransomware_detection.csv"
print(data_path)


C:\Users\richa\OneDrive\Documents\FYP2\datasets\static\ransomware_detection.csv


In [4]:
df = pd.read_csv(data_path)

print(df.columns.tolist())
df.head()


['FileName', 'md5Hash', 'Machine', 'DebugSize', 'DebugRVA', 'MajorImageVersion', 'MajorOSVersion', 'ExportRVA', 'ExportSize', 'IatVRA', 'MajorLinkerVersion', 'MinorLinkerVersion', 'NumberOfSections', 'SizeOfStackReserve', 'DllCharacteristics', 'ResourceSize', 'BitcoinAddresses', 'Benign']


,FileName,md5Hash,Machine,DebugSize,DebugRVA,MajorImageVersion,MajorOSVersion,ExportRVA,ExportSize,IatVRA,MajorLinkerVersion,MinorLinkerVersion,NumberOfSections,SizeOfStackReserve,DllCharacteristics,ResourceSize,BitcoinAddresses,Benign
0,0124e21d-018c-4ce0-92a3-b9e205a76bc0.dll,79755c51e413ed3c6be4635fd729a6e1,332,0,0,0,4,0,0,8192,8,0,3,1048576,34112,672,0,1
1,05c8318f98a5d301d80000009c316005.vertdll.dll,95e19f3657d34a432eada93221b0ea16,34404,84,121728,10,10,126576,4930,0,14,10,8,262144,16864,1024,0,1
2,06054fba-5619-4a86-a861-ffb0464bef5d.dll,85c32641d77a54e19ba8ea4ab305c791,332,0,0,0,4,0,0,8192,8,0,3,1048576,34112,672,0,1
3,075822ac99a5d301660400009c316005.adhapi.dll,62e3b959d982ef534b66f819fe15f085,34404,84,19904,10,10,21312,252,18160,14,10,6,262144,16736,1040,0,1
4,090607dd9ba5d301ca0900009c316005.SensorsNative...,ae38c5f7d313ad0ff3bfb8826476767f,34404,84,97728,10,10,105792,1852,70592,14,10,7,262144,16736,1096,0,1


In [6]:
import sys
from pathlib import Path
import pandas as pd

# Allow import of src.*
ROOT = Path("..").resolve()
sys.path.append(str(ROOT))

# Now import
from src.pe_static_extractor import extract_pe_features
from src.static_model import predict_static, get_static_feature_names

# EXE folder
EXE_DIR = Path(r"C:\Users\richa\source\repos\test_exe\test_exe")

files = [
    EXE_DIR / "hello.exe",
    EXE_DIR / "loop.exe",
    EXE_DIR / "sum.exe",
]

for f in files:
    print("\n====", f.name, "====")
    raw = f.read_bytes()

    # extract static PE features
    df_feats = extract_pe_features(raw)

    # predict using your XGBoost static model
    probs, preds, thr, reasons = predict_static(df_feats)

    print("Static prob:", float(probs[0]))
    print("Prediction (1=malicious):", int(preds[0]))
    print("Threshold:", thr)
    print("Top reasons:", reasons[:5])



==== hello.exe ====
Static prob: 0.07843402028083801
Prediction (1=malicious): 0
Threshold: 0.78
Top reasons: ['This file’s characteristics look unusual compared to safe files.']

==== loop.exe ====
Static prob: 0.07843402028083801
Prediction (1=malicious): 0
Threshold: 0.78
Top reasons: ['This file’s characteristics look unusual compared to safe files.']

==== sum.exe ====
Static prob: 0.07843402028083801
Prediction (1=malicious): 0
Threshold: 0.78
Top reasons: ['This file’s characteristics look unusual compared to safe files.']


In [5]:
import sys, os

# Point this to your project root
project_root = r"C:\Users\richa\OneDrive\Documents\FYP2"

if project_root not in sys.path:
    sys.path.append(project_root)


import pandas as pd
from src.behav_model import predict_behav

df = pd.read_csv(r"C:\Users\richa\OneDrive\Documents\FYP2\datasets\behavioral\behavior_features_agg.csv")

probs, preds, thr, reasons = predict_behav(df)

print("Behavioural probabilities:", probs)
print("Predictions:", preds)
print("Threshold:", thr)
print("Reasons:")
for r in reasons:
    print("-", r)


Behavioural probabilities: [1.]
Predictions: [1]
Threshold: 0.6
Reasons:
- Unusually high proportion of file-creation events, consistent with mass file modification or encryption behaviour.
- Network connection events detected, suggesting potential propagation or command-and-control communication.
- Multiple process creation events observed, which may indicate process spawning or code injection behaviour.


In [6]:
import pandas as pd

df = pd.read_parquet(r"C:\Users\richa\OneDrive\Documents\FYP2\data_processed\static_baseline.parquet")
print(df.columns)
print(df.head())

Index(['FileName', 'md5Hash', 'Machine', 'DebugSize', 'DebugRVA',
       'MajorImageVersion', 'MajorOSVersion', 'ExportRVA', 'ExportSize',
       'IatVRA', 'MajorLinkerVersion', 'MinorLinkerVersion',
       'NumberOfSections', 'SizeOfStackReserve', 'DllCharacteristics',
       'ResourceSize', 'BitcoinAddresses', 'Benign'],
      dtype='object')
                                            FileName  \
0           0124e21d-018c-4ce0-92a3-b9e205a76bc0.dll   
1       05c8318f98a5d301d80000009c316005.vertdll.dll   
2           06054fba-5619-4a86-a861-ffb0464bef5d.dll   
3        075822ac99a5d301660400009c316005.adhapi.dll   
4  090607dd9ba5d301ca0900009c316005.SensorsNative...   

                            md5Hash  Machine  DebugSize  DebugRVA  \
0  79755c51e413ed3c6be4635fd729a6e1      332          0         0   
1  95e19f3657d34a432eada93221b0ea16    34404         84    121728   
2  85c32641d77a54e19ba8ea4ab305c791      332          0         0   
3  62e3b959d982ef534b66f819fe15f085    3

In [7]:
import pandas as pd

dfb = pd.read_parquet(r"C:\Users\richa\OneDrive\Documents\FYP2\data_processed\behav_baseline.parquet")
print(dfb.columns)
print(dfb.head())


Index(['Time', 'Protocol', 'Flag', 'Family', 'Clusters', 'SeedAddress',
       'ExpAddress', 'BTC', 'USD', 'Netflow_Bytes', 'IPaddress', 'Threats',
       'Port', 'Prediction'],
      dtype='object')
   Time Protocol Flag    Family  Clusters SeedAddress ExpAddress  BTC  USD  \
0    50      TCP    A  WannaCry         1    1DA11mPS   1BonuSr7    1  500   
1    40      TCP    A  WannaCry         1    1DA11mPS   1BonuSr7    1  504   
2    30      TCP    A  WannaCry         1    1DA11mPS   1BonuSr7    1  508   
3    20      TCP    A  WannaCry         1    1DA11mPS   1BonuSr7    1  512   
4    57      TCP    A  WannaCry         1    1DA11mPS   1BonuSr7    1  516   

   Netflow_Bytes IPaddress Threats  Port Prediction  
0              5         A   Bonet  5061         SS  
1              8         A   Bonet  5061         SS  
2              7         A   Bonet  5061         SS  
3             15         A   Bonet  5061         SS  
4              9         A   Bonet  5061         SS  


In [8]:
import pandas as pd

dfb = pd.read_parquet(r"C:\Users\richa\OneDrive\Documents\FYP2\data_processed\behav_baseline.parquet")
print(dfb["Family"].value_counts())

# Using our label rule: y = 1 if Family != "Benign"
y = (dfb["Family"] != "Benign").astype(int)
print("\nLabel counts (y):")
print(y.value_counts())


Family
Locky               25062
SamSam              19657
WannaCry            16110
JigSaw              13712
Flyper              12014
DMALocker           11360
APT                  9730
CryptXXX             9335
Razy                 7862
Globe                7373
EDA2                 6054
TowerWeb             4381
Cryptohitman         4134
NoobCrypt            1248
CryptoLocker          788
CryptoLocker2015      150
Globev3                73
Name: count, dtype: int64

Label counts (y):
Family
1    149043
Name: count, dtype: int64


In [9]:
import pandas as pd
from pathlib import Path

project_root = Path(r"C:\Users\richa\OneDrive\Documents\FYP2")

behav_val_df = X_val.copy()
behav_val_df["label"] = y_val  # 1 = malicious, 0 = benign

out_path = project_root / "data_processed" / "behav_val.parquet"
behav_val_df.to_parquet(out_path, index=False)

print("Saved validation dataset to:", out_path)


NameError: name 'X_val' is not defined